In [ ]:
import pandas as pd
#dfC = pd.read_csv("non-cliques.csv", delimiter=" ", header=None)
dfC = pd.read_csv("top700imdb.txt", header=None)

non_cliques = dfC[0].values
non_cliques
# non_cliques = []
# for index, row in dfC.iterrows():
#     i = 0
#     for col in range(0, 12):
#         try:
#             if row[col][0] == "t":
#                 non_cliques.append(row[col])
#         except:
#             continue
# print(len(non_cliques))           
# blondieTitles = ["tt0038368","tt0039201","tt0040174","tt0040174","tt0029927","tt0031106","tt0031108",
#                 "tt0031108","tt0032261","tt0032262", "tt0032263", "tt0033403"]
# for title in blondieTitles:
#     non_cliques.append(title)
# len(non_cliques)

In [ ]:

import numpy as np
#Add RiffTrax and Wrestling perhaps
df1 = pd.read_csv("movie.statistics.csv")
df1.index = df1["Unnamed: 0"]
df2 = pd.read_csv("nodeStats.csv")
df2.index = df2["title"]
df = df1.join(df2)

df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
cutoffValue = df[df["cluster"]==4].sort_values("cluster.value", ascending=False).head(200)["cluster.value"].values[-1]
cutoffTwo = df[df["cluster"]==2].sort_values("cluster.value", ascending=False).head(100)["cluster.value"].values[-1]
cutoffSeven = df[df["cluster"]==7].sort_values("cluster.value", ascending=False).head(100)["cluster.value"].values[-1]

def flagClique(row):
    cluster = row["cluster"]
    value = row["cluster.value"]
    title = row["Unnamed: 0"]
    
    if cluster in [3, 5]:
        return 1
    elif cluster==2:
        if value < cutoffTwo:
            return None
        else:
            return 1
    elif cluster==7:
        if value < cutoffSeven:
            return None
        else:
            return 1
       
    elif title in non_cliques:
        return 0
    elif cluster==4:
        if value < cutoffValue:
            return None
        else:
            return 0
    else:
        return None
df["cliqueFlag"] = df.apply (lambda row: flagClique(row), axis=1)
dfTrain = df.dropna()

dfTrain

In [82]:
from sklearn.linear_model import LogisticRegression
dfTrain = dfTrain[~dfTrain.isin([np.nan, np.inf, -np.inf]).any(1)]
clf = LogisticRegression(random_state=0, max_iter=1000).fit(dfTrain[["degrees", "degress.from.tam", "coreness", "triangles", "degMode", "degMean", "degLogSD", "degLogSkew", "degLogKurt"]], dfTrain["cliqueFlag"])

In [83]:
print(clf.coef_, clf.intercept_)
print(clf.score(dfTrain[["degrees", "degress.from.tam", "coreness", "triangles", "degMode", "degMean", "degLogSD", "degLogSkew", "degLogKurt"]], dfTrain["cliqueFlag"]))

[[-2.57630170e-02  4.38438663e-02  3.32003578e-02 -1.24979839e-04
  -2.23444385e-01  2.10885385e+00 -1.32987434e+00  2.49286406e+00
   4.33868275e-01]] [-0.96108656]
0.8875


In [ ]:
df["predictedClique"] = clf.predict(df[["degrees", "degress.from.tam", "coreness", "triangles", "degMode", "degMean", "degLogSD", "degLogSkew", "degLogKurt"]])
df

In [ ]:
dfTrain[dfTrain["Unnamed: 0"]=="tt0038368"]
df[df["Unnamed: 0"]=="tt0038368"]

In [88]:
pred_non_clique = df[df["predictedClique"]==0]["Unnamed: 0"].values
pred_non_clique

array(['tt0002337', 'tt0002483', 'tt0002605', ..., 'tt9896768',
       'tt9898328', 'tt9903348'], dtype=object)

In [89]:
pd.DataFrame(pred_non_clique, columns=["pred_non_clique_titles"]).to_csv("pred_non_clique_titles.csv")

In [7]:
pred_non_clique

array(['tt0001230', 'tt0001498', 'tt0002337', ..., 'tt9730696',
       'tt9887154', 'tt9898328'], dtype=object)